In [1]:
%cd /Users/pg/OneDrive/Duke Assigment/Job Seeking/Collection_DS_takehome_challenges/data

/Users/pg/OneDrive/Duke Assigment/Job Seeking/Collection_DS_takehome_challenges/data


In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

%matplotlib inline

In [3]:
referral = pd.read_csv('referral.csv')
referral['date'] = pd.to_datetime(referral['date'])

In [4]:
referral.head()

,user_id,date,country,money_spent,is_referral,device_id
0,2,2015-10-03,FR,65,0,EVDCJTZMVMJDG
1,3,2015-10-03,CA,54,0,WUBZFTVKXGQQX
2,6,2015-10-03,FR,35,0,CBAPCJRTFNUJG
3,7,2015-10-03,UK,73,0,PRGXJZAJKMXRH
4,7,2015-10-03,MX,35,0,PRGXJZAJKMXRH


In [5]:
referral.info()
#no null value
#data type is appropriate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 6 columns):
user_id        97341 non-null int64
date           97341 non-null datetime64[ns]
country        97341 non-null object
money_spent    97341 non-null int64
is_referral    97341 non-null int64
device_id      97341 non-null object
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.5+ MB


In [11]:
referral.describe()

,user_id,money_spent,is_referral
count,97341.000000,97341.000000,97341.000000
mean,6354.801029,44.692360,0.287823
std,5682.991186,22.791839,0.452751
min,1.000000,10.000000,0.000000
25%,2020.000000,27.000000,0.000000
50%,4053.000000,42.000000,0.000000
75%,10286.000000,59.000000,1.000000
max,20000.000000,220.000000,1.000000


In [6]:
start_date = pd.to_datetime('2015-10-31')

**Q1: Can you estimate the impact the program had on the site?**

In [7]:
start_date

Timestamp('2015-10-31 00:00:00')

In [8]:
referral.head()

,user_id,date,country,money_spent,is_referral,device_id
0,2,2015-10-03,FR,65,0,EVDCJTZMVMJDG
1,3,2015-10-03,CA,54,0,WUBZFTVKXGQQX
2,6,2015-10-03,FR,35,0,CBAPCJRTFNUJG
3,7,2015-10-03,UK,73,0,PRGXJZAJKMXRH
4,7,2015-10-03,MX,35,0,PRGXJZAJKMXRH


In [15]:
#what metrics is worth considering?
# # of purchase, total money spent, # of customers on eacy day

In [9]:
def count_spent(df):
    d = {}
    d['n_purchase'] = df.shape[0]
    d['total_spent'] = df.money_spent.sum()
    d['n_customers']=len(df.user_id.unique())
    return pd.Series(d)

In [10]:
def daily_statistics(df):
    grpby_day = df.groupby('date').apply(count_spent)
    
    grpby_before = grpby_day.loc[grpby_day.index < start_date, :]
    grpby_after = grpby_day.loc[grpby_day.index >= start_date, :]
    
    d = []
    columns = ['total_spent', 'n_purchase', 'n_customers']
    
    for col in columns:
        pre_data = grpby_before.loc[:, col]
        pre_mean = pre_data.mean()
        
        after_data = grpby_after.loc[:, col]
        after_mean = after_data.mean()
        
        t_test = ss.ttest_ind(pre_data, after_data, equal_var= False)[1]
        p_value = t_test/2
        d.append({'mean_pre': pre_mean, 'mean_post': after_mean, 'mean_diff': after_mean-pre_mean, 'pvalue':p_value})
        
    return pd.DataFrame(d, index = columns)
        

In [11]:
daily_statistics(referral)

,mean_diff,mean_post,mean_pre,pvalue
total_spent,12057.392857,83714.392857,71657.000000,0.135194
n_purchase,94.964286,1785.714286,1690.750000,0.348257
n_customers,302.500000,1686.964286,1384.464286,0.059545


**According to the data, although the 'total_spent', 'n_purchase', 'n_customers' all increased, the increase is not stistically significant**

**Hypothesis test grouped by country**

In [13]:
daily_stat_by_country = referral.groupby('country').apply(daily_statistics)

In [14]:
daily_stat_by_country

mean_diff     mean_post      mean_pre    pvalue
country                                                               
CA      total_spent   412.000000   7880.428571   7468.428571  0.351704
        n_purchase    -17.142857    160.000000    177.142857  0.233985
        n_customers   -14.107143    159.178571    173.285714  0.268256
CH      total_spent  -512.428571   1023.892857   1536.321429  0.006941
        n_purchase     -9.750000     17.071429     26.821429  0.003072
        n_customers    -9.642857     17.071429     26.714286  0.003142
DE      total_spent -1842.785714   8013.964286   9856.750000  0.081459
        n_purchase    -68.107143    164.035714    232.142857  0.011798
        n_customers   -61.714286    163.250000    224.964286  0.015665
ES      total_spent  2011.928571   8660.571429   6648.642857  0.037522
        n_purchase     37.892857    194.500000    156.607143  0.072638
        n_customers    39.821429    193.214286    153.392857  0.057954
FR      total_spent  3249.750000  13635.000000  10385.250000  0.031843
        n_purchase     61.571429    305.714286    244.142857  0.058996
        n_customers    66.035714    302.535714    236.500000  0.041124
IT      total_spent  2541.857143  10193.428571   7651.571429  0.025730
        n_purchase     47.071429    227.928571    180.857143  0.057454
        n_customers    49.571429    226.107143    176.535714  0.043911
MX      total_spent  2057.750000   7033.214286   4975.464286  0.009670
        n_purchase     37.535714    164.000000    126.464286  0.032430
        n_customers    38.714286    163.107143    124.392857  0.026203
UK      total_spent  2982.892857  14196.428571  11213.535714  0.048490
        n_purchase     24.750000    289.035714    264.285714  0.261183
        n_customers    30.750000    286.321429    255.571429  0.204398
US      total_spent  1156.428571  13077.464286  11921.035714  0.248874
        n_purchase    -18.857143    263.428571    282.285714  0.307801
        n_customers   -12.071429    261.107143    273.178571  0.368860

In [19]:
daily_stat_by_country.xs('total_spent', level = 1).sort_values(by = 'pvalue')
#.xs: This method takes a key argument to select data at a particular level of a MultiIndex.

,mean_diff,mean_post,mean_pre,pvalue
country,,,,
CH,-512.428571,1023.892857,1536.321429,0.006941
MX,2057.750000,7033.214286,4975.464286,0.009670
IT,2541.857143,10193.428571,7651.571429,0.025730
FR,3249.750000,13635.000000,10385.250000,0.031843
ES,2011.928571,8660.571429,6648.642857,0.037522
UK,2982.892857,14196.428571,11213.535714,0.048490
DE,-1842.785714,8013.964286,9856.750000,0.081459
US,1156.428571,13077.464286,11921.035714,0.248874
CA,412.000000,7880.428571,7468.428571,0.351704


In [23]:
daily_stat_by_country.xs('n_customers', level = 1).sort_values(by = 'pvalue')

,mean_diff,mean_post,mean_pre,pvalue
country,,,,
CH,-9.642857,17.071429,26.714286,0.003142
DE,-61.714286,163.250000,224.964286,0.015665
MX,38.714286,163.107143,124.392857,0.026203
FR,66.035714,302.535714,236.500000,0.041124
IT,49.571429,226.107143,176.535714,0.043911
ES,39.821429,193.214286,153.392857,0.057954
UK,30.750000,286.321429,255.571429,0.204398
CA,-14.107143,159.178571,173.285714,0.268256
US,-12.071429,261.107143,273.178571,0.368860


In [24]:
daily_stat_by_country.xs('n_purchase', level = 1).sort_values(by = 'pvalue')

,mean_diff,mean_post,mean_pre,pvalue
country,,,,
CH,-9.750000,17.071429,26.821429,0.003072
DE,-68.107143,164.035714,232.142857,0.011798
MX,37.535714,164.000000,126.464286,0.032430
IT,47.071429,227.928571,180.857143,0.057454
FR,61.571429,305.714286,244.142857,0.058996
ES,37.892857,194.500000,156.607143,0.072638
CA,-17.142857,160.000000,177.142857,0.233985
UK,24.750000,289.035714,264.285714,0.261183
US,-18.857143,263.428571,282.285714,0.307801


# Country based conclusion#

**CH, DE has significant decrease**

**MX, IT, FR, ES has significant increase**

**UK, US, CA show no significant change**

# Can you estimate the impact the program had on the site?#

**The program has no overall effect, but has some effect according to countries**

# Based on the data, what would you suggest to do as a next step?#

**Since the analysis is not very solid, I suggest conduct A/B testing to see if the program has significant effect.**

**For the countries showing different changes, launch a study to analyze why. E.g, CH and DE show a negative change and so it's worth considering.**

# The referral program wasn't really tested in a rigorous way. It simply started on a given day for all users and you are drawing conclusions by looking at the data before and after the test started. What kinds of risks this approach presents? Can you think of a better way to test the referral program and measure its impact?#

**The approach is not a solid A/B testing. It simply started on a given day for all users and the could be other differences between control group and test group. For example, there may be some special holiday after Oct. 31.**

**In order to imporove the test, we need to perform a more careful A/B testing. For example:**

**1. Use the same period of time**

**2. Randomly split customers in two groups and let only one group know about the user referral program**

**3. Run the experinment and perform the t-test to see whether some sale performance metrics are changed significantly.**